<a href="https://colab.research.google.com/github/waqasde/AIQ2_Waqas_Ahmad_PIAIC139128/blob/main/Agentic_AI_Chatbot_Project_03_Browsing_AI_Agent_Your_Next_Gen_Answer_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain_anthropic

In [2]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic AI Chatbot Prototype 1"

gemini_api_key = userdata.get('GEMINI_API_KEY')

In [3]:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain_google_genai

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_retries=2,
    api_key=gemini_api_key
)

In [5]:
%%capture --no-stderr
%pip install -U tavily-python langchain_community

In [6]:
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [ ]:
from typing import Annotated, Literal
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AIMessage, ToolMessage, SystemMessage
from pydantic import BaseModel
from typing_extensions import TypedDict
from concurrent.futures import ThreadPoolExecutor
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]
    ask_human: bool
    ask_user: bool
    long_term_memory: dict  # To personalize answers based on prior interactions

class RequestAssistance(BaseModel):
    """Escalate the conversation to an expert."""
    request: str

# Define the system message to enforce browsing assistant context
system_prompt = SystemMessage(
    content=(
        "You are a browsing AI assistant. Your job is to help users break down complex queries, search online for \
        relevant information, and synthesize results into concise, actionable answers. If a query cannot be addressed, \
        politely respond and guide the user on how they can refine their question."
    )
)

# Initialize tools
tool = TavilySearchResults(max_results=5)
tools = [tool]

def decompose_query(query: str):
    """Break down a complex query into smaller, actionable sub-queries."""
    # Example logic for query decomposition (can be customized further)
    if " and " in query.lower():
        return [sub.strip() for sub in query.split(" and ")]
    return [query]

def execute_sub_queries(tool, sub_tasks):
    """Execute sub-tasks in parallel."""
    results = []
    with ThreadPoolExecutor() as executor:
        future_to_task = {executor.submit(tool.invoke, task): task for task in sub_tasks}
        for future in future_to_task:
            try:
                results.append(future.result())
            except Exception as e:
                results.append(f"Error fetching results for {future_to_task[future]}: {e}")
    return results

def synthesize_results(sub_tasks, results):
    """Combine results into a concise and clear answer."""
    synthesized_response = "\n".join(
        [f"For '{task}', I found: {result}" for task, result in zip(sub_tasks, results)]
    )
    return synthesized_response

def chatbot(state: State):
    # Inject the system prompt if it's not already included
    if not any(isinstance(msg, SystemMessage) for msg in state["messages"]):
        state["messages"].insert(0, system_prompt)

    user_message = state["messages"][-1].content if state["messages"] else ""

    # Decompose query
    sub_tasks = decompose_query(user_message)

    # Inform user of progress
    progress_messages = [
        AIMessage(content=f"Decomposing query into tasks: {', '.join(sub_tasks)}"),
        AIMessage(content="Executing searches in parallel. Please wait...")
    ]
    state["messages"].extend(progress_messages)

    # Execute sub-queries
    results = execute_sub_queries(tool, sub_tasks)

    # Synthesize results
    final_response = synthesize_results(sub_tasks, results)

    # Add synthesized response to messages
    return {
        "messages": state["messages"] + [AIMessage(content=final_response)],
        "ask_human": False,
        "ask_user": False,
    }

# Build the state graph
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=[tool]))

def human_node(state: State):
    return {
        "messages": [
            AIMessage(content="A human assistant has been notified and will assist you shortly.")
        ],
        "ask_human": False,
    }

def user_interaction_node(state: State):
    return {
        "messages": [
            AIMessage(content="Thanks for the clarification. Let me continue assisting you."),
        ],
        "ask_user": False,
    }

graph_builder.add_node("human", human_node)
graph_builder.add_node("user_interaction", user_interaction_node)

def select_next_node(state: State):
    if state["ask_human"]:
        return "human"
    if state["ask_user"]:
        return "user_interaction"
    return tools_condition(state)

graph_builder.add_conditional_edges(
    "chatbot",
    select_next_node,
    {"human": "human", "user_interaction": "user_interaction", "tools": "tools", END: END},
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("human", "chatbot")
graph_builder.add_edge("user_interaction", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(
    checkpointer=memory,
    interrupt_before=["human", "user_interaction"],
)

def stream_graph_updates(user_input: str):
    """
    Streams updates from the chatbot graph based on user input.

    Args:
        user_input (str): The input provided by the user.
    """
    # Initialize state with default values
    state = {
        "messages": [("user", user_input)],
        "ask_human": False,
        "ask_user": False,
        "long_term_memory": {},  # Initialize long-term memory
    }

    # Configuration for graph streaming
    config = {"configurable": {"thread_id": "1"}}

    # Validate user input before processing
    if not user_input.strip():
        print("Assistant: It seems you didn't enter anything. Please ask a detailed question.")
        return

    try:
        # Stream responses from the graph
        for event in graph.stream(state, config):
            for value in event.values():
                print("Assistant:", value["messages"][-1].content)
    except KeyError as e:
        print(f"KeyError: {e} - Ensure all required keys are initialized in the state.")
    except Exception as e:
        print(f"An error occurred: {e}")

def main():
    """
    Main function to run the interactive browsing AI chatbot.
    """
    print("Welcome to the Browsing AI Chatbot! Ask me complex questions, and I'll find the best answers for you.")
    print("Type 'quit', 'exit', or 'q' to end the conversation.")

    while True:
        try:
            # Get user input
            user_input = input("You: ").strip()

            # Exit condition
            if user_input.lower() in ["quit", "exit", "q"]:
                print("Goodbye! Hope I could assist you well.")
                break

            # Stream graph updates with user input
            stream_graph_updates(user_input)
        except KeyboardInterrupt:
            # Graceful exit on Ctrl+C
            print("\nGoodbye! Stay curious and informed.")
            break
        except Exception as e:
            # Handle unexpected errors
            print(f"An unexpected error occurred: {e}")
            break

if __name__ == "__main__":
    main()


Welcome to the Browsing AI Chatbot! Ask me complex questions, and I'll find the best answers for you.
Type 'quit', 'exit', or 'q' to end the conversation.
You: tell me near by best doctors
Assistant: For 'tell me near by best doctors', I found: [{'url': 'https://doctor.webmd.com/', 'content': 'Find Doctors Near You: Top Physician Directory Find Providers by Specialty Find Doctors and Dentists Near You Find Doctors and Dentists Near You You can also search by physician, practice, or hospital name Choose the healthcare that is right for you The WebMD Choice Awards is the only hospital recognition program based on the opinion of patients and health care providers. View all specialties ADD/ADHD Testing Find Doctors and Dentists Near You Find Doctors and Dentists Near You You can also search by physician, practice, or hospital name Find Doctors •Pain Medicine Doctors •Primary Care Doctors Find Providers by Specialty New York City doctors Los Angeles doctors Chicago doctors Policies About We